In [1]:
import pandas as pd
from eval import my_grid_search_cv
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# from tqdm import tqdm
# tqdm.pandas()

In [2]:
import os

if os.getcwd() != "/root/restaurants":
    os.chdir("..")
os.getcwd()

'/root/restaurants'

In [3]:
data = pd.read_csv("data/processed/data.csv")
data.head()

data["date"] = pd.to_datetime(data["date"])
data = data.rename(columns={"date": "visit_date"})


In [4]:
data

,store_id,visit_date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,...,is_nan_lag_holiday_flg_273,lag_area_genre_day_of_week_365,is_nan_lag_area_genre_day_of_week_365,visitors,lag_area_genre_holiday_flg_365,is_nan_lag_area_genre_holiday_flg_365,lag_day_of_week_365,is_nan_lag_day_of_week_365,lag_holiday_flg_365,is_nan_lag_holiday_flg_365
0,air_00a91d42b08b08d9,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,1,0.000000,1,24,0.000000,1,0.000000,1,0.000000,1
1,air_0241aa3964b7f861,2016-01-03,Sunday,1,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996,2016,1,...,1,0.000000,1,28,0.000000,1,0.000000,1,0.000000,1
2,air_034a3d5b40d5b1b1,2016-07-01,Friday,0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229,2016,7,...,1,0.000000,1,9,0.000000,1,0.000000,1,0.000000,1
3,air_036d4f1ee7285390,2016-07-01,Friday,0,Cafe/Sweets,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,2016,7,...,1,0.000000,1,10,0.000000,1,0.000000,1,0.000000,1
4,air_04341b588bde96cd,2016-01-01,Friday,1,Izakaya,Tōkyō-to Nerima-ku Toyotamakita,35.735623,139.651658,2016,1,...,1,0.000000,1,12,0.000000,1,0.000000,1,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_dbf64f1ce38c7442,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.411796,0,3,23.089882,0,25.078947,0,22.859375,0
252104,air_e57dd6884595f60d,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.414295,0,47,23.090176,0,24.730769,0,23.020690,0
252105,air_eb120e6d384a17a8,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.416789,0,35,23.090470,0,23.285714,0,22.698052,0
252106,air_eca5e0064dc9314a,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.419278,0,21,23.090765,0,19.785714,0,21.761905,0


In [5]:
# data = data[data["store_id"] == "air_00a91d42b08b08d9"]

X = data.drop(columns=["visitors"]).reset_index(drop=True)
y = data[["visitors"]].reset_index(drop=True)

for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[c] = X[c].astype('category')


In [ ]:
import numpy as np
num_cols = np.array(X.select_dtypes(include=['number']).columns)
num_cols

array(['holiday_flg', 'latitude', 'longitude', 'year', 'month', 'day',
       'lag_area_genre_day_of_week_21',
       'is_nan_lag_area_genre_day_of_week_21',
       'lag_area_genre_holiday_flg_21',
       'is_nan_lag_area_genre_holiday_flg_21', 'lag_day_of_week_21',
       'is_nan_lag_day_of_week_21', 'lag_holiday_flg_21',
       'is_nan_lag_holiday_flg_21', 'lag_area_genre_day_of_week_35',
       'is_nan_lag_area_genre_day_of_week_35',
       'lag_area_genre_holiday_flg_35',
       'is_nan_lag_area_genre_holiday_flg_35', 'lag_day_of_week_35',
       'is_nan_lag_day_of_week_35', 'lag_holiday_flg_35',
       'is_nan_lag_holiday_flg_35', 'lag_area_genre_day_of_week_61',
       'is_nan_lag_area_genre_day_of_week_61',
       'lag_area_genre_holiday_flg_61',
       'is_nan_lag_area_genre_holiday_flg_61', 'lag_day_of_week_61',
       'is_nan_lag_day_of_week_61', 'lag_holiday_flg_61',
       'is_nan_lag_holiday_flg_61', 'lag_area_genre_day_of_week_91',
       'is_nan_lag_area_genre_day_of_wee

In [7]:
param_grid = {        
    "n_estimators": [5, 10, 20, 30, 40, 50, 70],
    "learning_rate": [0.01, 0.05, 0.1, 0.5],
    "max_depth": [3, 4, 5],

    "enable_categorical": ["True"],
}

params = my_grid_search_cv(XGBRegressor, X, y, param_grid, n_jobs=1)

model = XGBRegressor(**params)

X = X.drop(columns=["visit_date"]).reset_index(drop=True)

scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

model.fit(X, y)

  0%|          | 0/84 [00:00<?, ?it/s]

 17%|█▋        | 14/84 [01:16<06:46,  5.81s/it]

: 

In [7]:
directory = "models/xgboost"
os.makedirs(directory, exist_ok=True)
model.save_model(directory + '/xgboost.json')

In [7]:
THREE_WEEKS = 21
FIVE_WEEKS = 35
TWO_MONTH = 61
ONE_QUARTER = 365 // 4
HALF_YEAR = 365 // 2
THREE_QUARTERS = 365 * 3 // 4
YEAR = 365

LAGS = [
    THREE_WEEKS,
    FIVE_WEEKS,
    TWO_MONTH,
    ONE_QUARTER,
    HALF_YEAR,
    THREE_QUARTERS,
    YEAR,
]


In [8]:
model = XGBRegressor()
model.load_model("models/xgboost/xgboost.json")

In [9]:
from tqdm.auto import tqdm

In [10]:
import math

def add_lag_last(df, lag, column):
    column_name = "lag_" + column + "_" + str(lag)

    temp = df[df["store_id"] == df.iloc[-1]["store_id"]]

    temp = temp[temp[column] == temp.iloc[-1][column]]

    start_date = df.iloc[-1]["date"]
    end_date = start_date - pd.Timedelta(days=lag)

    value = temp[(temp['date'] < start_date) & (temp['date'] > end_date)]["visitors"].mean()

    if math.isnan(value):
        df.at[df.index[-1], column_name] = 0
        df.at[df.index[-1], "is_nan_" + column_name] = 1
    else:
        df.at[df.index[-1], column_name] = value
        df.at[df.index[-1], "is_nan_" + column_name] = 0

    return df

def get_area_genre_lag_last(df, lag, column):
    column_name = "lag_area_genre_" + column + "_" + str(lag)
    
    area_genre_data = df[df["area_name"] == df.iloc[-1]["area_name"]]
    area_genre_data = area_genre_data[area_genre_data["genre_name"] == area_genre_data.iloc[-1]["genre_name"]]

    area_genre_data = area_genre_data[
        ["area_name", "genre_name", "date", "visitors", column]
    ]

    area_genre_data_mean = area_genre_data.groupby(by=["date"]).visitors.mean()
    
    area_genre_data = (
        area_genre_data.drop(columns=["visitors"])
        .merge(area_genre_data_mean, on=["date"], how="right")
        .drop_duplicates()
    )

    temp = area_genre_data[area_genre_data[column] == area_genre_data.iloc[-1][column]]

    start_date = df.iloc[-1]["date"]
    end_date = start_date - pd.Timedelta(days=lag)

    value = temp[(temp['date'] < start_date) & (temp['date'] >= end_date)]["visitors"].mean()

    if math.isnan(value):
        df.at[df.index[-1], column_name] = 0
        df.at[df.index[-1], "is_nan_" + column_name] = 1
    else:
        df.at[df.index[-1], column_name] = value
        df.at[df.index[-1], "is_nan_" + column_name] = 0

    return df


In [11]:
ID_SIZE = 20

store_info = pd.read_csv("data/raw/air_store_info.csv")
store_info = store_info.rename(
    columns={
        "air_store_id": "store_id",
        "air_genre_name": "genre_name",
        "air_area_name": "area_name",
    }   
)

date_info = pd.read_csv("data/raw/date_info.csv")
date_info = date_info.rename(columns={"calendar_date": "date"})
date_info["date"] = date_info["date"].astype("string")

In [28]:
def make_prediction(row):
    global data
    new_row = pd.DataFrame(row).T
    new_row["store_id"] = row["id"][:ID_SIZE]
    new_row["date"] = row["id"][ID_SIZE + 1:]
    new_row = new_row.drop(columns="id")

    new_row = pd.merge(new_row, date_info, on="date")
    new_row = pd.merge(new_row, store_info, on="store_id", how="left")
    
    new_row["date"] = pd.to_datetime(new_row["date"])
    new_row["year"] = new_row["date"].dt.year
    new_row["month"] = new_row["date"].dt.month
    new_row["day"] = new_row["date"].dt.day

    data = pd.concat([data, new_row]).reset_index(drop=True)

    for lag in LAGS:
        data = get_area_genre_lag_last(data, lag, "day_of_week")
        data = get_area_genre_lag_last(data, lag, "holiday_flg")
        data = add_lag_last(data, lag, "day_of_week")
        data = add_lag_last(data, lag, "holiday_flg")


    new_row = data.tail(1)

    for c in new_row.columns:
        col_type = new_row[c].dtype
        if col_type == 'object' or col_type.name == 'category':
            new_row[c] = new_row[c].astype('category')

    new_row = new_row.drop(columns=["date", "visitors"])

    new_row[num_cols] = scaler.transform(new_row[num_cols])
    
    pred = 1
    pred = model.predict(new_row)
    pred = max(0, pred)

    row["visitors"] = pred

    data.at[data.index[-1], "visitors"] = pred
   
    return row

In [36]:
submission = pd.read_csv("data/raw/sample_submission.csv").head(100)
data = pd.read_csv("data/processed/data.csv")
# data = data.head(100)
data["date"] = pd.to_datetime(data["date"])
data

,store_id,date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,...,is_nan_lag_holiday_flg_273,lag_area_genre_day_of_week_365,is_nan_lag_area_genre_day_of_week_365,visitors,lag_area_genre_holiday_flg_365,is_nan_lag_area_genre_holiday_flg_365,lag_day_of_week_365,is_nan_lag_day_of_week_365,lag_holiday_flg_365,is_nan_lag_holiday_flg_365
0,air_00a91d42b08b08d9,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,1,0.000000,1,24,0.000000,1,0.000000,1,0.000000,1
1,air_0241aa3964b7f861,2016-01-03,Sunday,1,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996,2016,1,...,1,0.000000,1,28,0.000000,1,0.000000,1,0.000000,1
2,air_034a3d5b40d5b1b1,2016-07-01,Friday,0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229,2016,7,...,1,0.000000,1,9,0.000000,1,0.000000,1,0.000000,1
3,air_036d4f1ee7285390,2016-07-01,Friday,0,Cafe/Sweets,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,2016,7,...,1,0.000000,1,10,0.000000,1,0.000000,1,0.000000,1
4,air_04341b588bde96cd,2016-01-01,Friday,1,Izakaya,Tōkyō-to Nerima-ku Toyotamakita,35.735623,139.651658,2016,1,...,1,0.000000,1,12,0.000000,1,0.000000,1,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_dbf64f1ce38c7442,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.411796,0,3,23.089882,0,25.078947,0,22.859375,0
252104,air_e57dd6884595f60d,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.414295,0,47,23.090176,0,24.730769,0,23.020690,0
252105,air_eb120e6d384a17a8,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.416789,0,35,23.090470,0,23.285714,0,22.698052,0
252106,air_eca5e0064dc9314a,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0,22.419278,0,21,23.090765,0,19.785714,0,21.761905,0


In [37]:
data.columns

Index(['store_id', 'date', 'day_of_week', 'holiday_flg', 'genre_name',
       'area_name', 'latitude', 'longitude', 'year', 'month', 'day',
       'lag_area_genre_day_of_week_21', 'is_nan_lag_area_genre_day_of_week_21',
       'lag_area_genre_holiday_flg_21', 'is_nan_lag_area_genre_holiday_flg_21',
       'lag_day_of_week_21', 'is_nan_lag_day_of_week_21', 'lag_holiday_flg_21',
       'is_nan_lag_holiday_flg_21', 'lag_area_genre_day_of_week_35',
       'is_nan_lag_area_genre_day_of_week_35', 'lag_area_genre_holiday_flg_35',
       'is_nan_lag_area_genre_holiday_flg_35', 'lag_day_of_week_35',
       'is_nan_lag_day_of_week_35', 'lag_holiday_flg_35',
       'is_nan_lag_holiday_flg_35', 'lag_area_genre_day_of_week_61',
       'is_nan_lag_area_genre_day_of_week_61', 'lag_area_genre_holiday_flg_61',
       'is_nan_lag_area_genre_holiday_flg_61', 'lag_day_of_week_61',
       'is_nan_lag_day_of_week_61', 'lag_holiday_flg_61',
       'is_nan_lag_holiday_flg_61', 'lag_area_genre_day_of_week_91',

In [34]:
data[data["store_id"] == "air_00a91d42b08b08d9"]

,store_id,date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,...,is_nan_lag_holiday_flg_273,lag_area_genre_day_of_week_365,is_nan_lag_area_genre_day_of_week_365,visitors,lag_area_genre_holiday_flg_365,is_nan_lag_area_genre_holiday_flg_365,lag_day_of_week_365,is_nan_lag_day_of_week_365,lag_holiday_flg_365,is_nan_lag_holiday_flg_365
0,air_00a91d42b08b08d9,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,1.0,0.000000,1.0,24,0.000000,1.0,0.000000,1.0,0.000000,1.0
976,air_00a91d42b08b08d9,2016-07-02,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,0.0,0.000000,1.0,47,24.250000,0.0,0.000000,1.0,24.000000,0.0
2196,air_00a91d42b08b08d9,2016-07-04,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,0.0,0.000000,1.0,3,32.444444,0.0,0.000000,1.0,35.500000,0.0
2928,air_00a91d42b08b08d9,2016-07-05,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,0.0,0.000000,1.0,7,26.833333,0.0,0.000000,1.0,24.666667,0.0
3660,air_00a91d42b08b08d9,2016-07-06,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,0.0,0.000000,1.0,32,23.466667,0.0,0.000000,1.0,20.250000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252133,air_00a91d42b08b08d9,2017-05-18,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,5,...,0.0,23.808162,0.0,21.03909,20.409280,0.0,33.978470,0.0,22.372305,0.0
252134,air_00a91d42b08b08d9,2017-05-19,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,5,...,0.0,21.951825,0.0,21.03909,20.411408,0.0,22.212029,0.0,22.367035,0.0
252135,air_00a91d42b08b08d9,2017-05-20,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,5,...,0.0,19.911865,0.0,21.03909,20.413521,0.0,20.588226,0.0,22.361807,0.0
252136,air_00a91d42b08b08d9,2017-05-21,Sunday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,5,...,0.0,17.348961,0.0,21.03909,20.415620,0.0,27.631271,0.0,22.356620,0.0


In [35]:
model

XGBRegressor(base_score='2.097376E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['c', 'c', 'float', 'c', 'c', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float', 'float...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [38]:
submission = submission.sample(30)
submission.progress_apply(lambda row: make_prediction(row), axis=1)

  0%|          | 0/30 [00:00<?, ?it/s]

,id,visitors
27,air_00a91d42b08b08d9_2017-05-20,21.03909
85,air_0241aa3964b7f861_2017-04-30,21.03909
63,air_0164b9927d20bcc3_2017-05-17,21.03909
83,air_0241aa3964b7f861_2017-04-28,21.03909
52,air_0164b9927d20bcc3_2017-05-06,21.03909
77,air_0164b9927d20bcc3_2017-05-31,21.03909
11,air_00a91d42b08b08d9_2017-05-04,21.03909
31,air_00a91d42b08b08d9_2017-05-24,21.03909
3,air_00a91d42b08b08d9_2017-04-26,21.03909
79,air_0241aa3964b7f861_2017-04-24,21.03909


In [23]:
make_prediction(submission.iloc[0])

/tmp/ipykernel_116404/636511531.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row[c] = new_row[c].astype('category')
/tmp/ipykernel_116404/636511531.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row[c] = new_row[c].astype('category')
/tmp/ipykernel_116404/636511531.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,store_id,date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,...,is_nan_lag_holiday_flg_273,lag_area_genre_day_of_week_365,is_nan_lag_area_genre_day_of_week_365,visitors,lag_area_genre_holiday_flg_365,is_nan_lag_area_genre_holiday_flg_365,lag_day_of_week_365,is_nan_lag_day_of_week_365,lag_holiday_flg_365,is_nan_lag_holiday_flg_365
0,air_00a91d42b08b08d9,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,1.0,0.000000,1.0,24,0.000000,1.0,0.000000,1.0,0.000000,1.0
1,air_0241aa3964b7f861,2016-01-03,Sunday,1,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996,2016,1,...,1.0,0.000000,1.0,28,0.000000,1.0,0.000000,1.0,0.000000,1.0
2,air_034a3d5b40d5b1b1,2016-07-01,Friday,0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229,2016,7,...,1.0,0.000000,1.0,9,0.000000,1.0,0.000000,1.0,0.000000,1.0
3,air_036d4f1ee7285390,2016-07-01,Friday,0,Cafe/Sweets,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,2016,7,...,1.0,0.000000,1.0,10,0.000000,1.0,0.000000,1.0,0.000000,1.0
4,air_04341b588bde96cd,2016-01-01,Friday,1,Izakaya,Tōkyō-to Nerima-ku Toyotamakita,35.735623,139.651658,2016,1,...,1.0,0.000000,1.0,12,0.000000,1.0,0.000000,1.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252104,air_e57dd6884595f60d,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.414295,0.0,47,23.090176,0.0,24.730769,0.0,23.020690,0.0
252105,air_eb120e6d384a17a8,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.416789,0.0,35,23.090470,0.0,23.285714,0.0,22.698052,0.0
252106,air_eca5e0064dc9314a,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.419278,0.0,21,23.090765,0.0,19.785714,0.0,21.761905,0.0
252107,air_ef47430bcd6f6a89,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.421763,0.0,10,23.091059,0.0,19.000000,0.0,23.113821,0.0


In [26]:
make_prediction(submission.iloc[1])

/tmp/ipykernel_114420/953428861.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["visitors"] = pred


,store_id,date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,...,is_nan_lag_holiday_flg_273,lag_area_genre_day_of_week_365,is_nan_lag_area_genre_day_of_week_365,visitors,lag_area_genre_holiday_flg_365,is_nan_lag_area_genre_holiday_flg_365,lag_day_of_week_365,is_nan_lag_day_of_week_365,lag_holiday_flg_365,is_nan_lag_holiday_flg_365
0,air_00a91d42b08b08d9,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,1.0,0.000000,1.0,24,0.000000,1.0,0.000000,1.0,0.000000,1.0
1,air_0241aa3964b7f861,2016-01-03,Sunday,1,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996,2016,1,...,1.0,0.000000,1.0,28,0.000000,1.0,0.000000,1.0,0.000000,1.0
2,air_034a3d5b40d5b1b1,2016-07-01,Friday,0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229,2016,7,...,1.0,0.000000,1.0,9,0.000000,1.0,0.000000,1.0,0.000000,1.0
3,air_036d4f1ee7285390,2016-07-01,Friday,0,Cafe/Sweets,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,2016,7,...,1.0,0.000000,1.0,10,0.000000,1.0,0.000000,1.0,0.000000,1.0
4,air_04341b588bde96cd,2016-01-01,Friday,1,Izakaya,Tōkyō-to Nerima-ku Toyotamakita,35.735623,139.651658,2016,1,...,1.0,0.000000,1.0,12,0.000000,1.0,0.000000,1.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252105,air_eb120e6d384a17a8,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.416789,0.0,35,23.090470,0.0,23.285714,0.0,22.698052,0.0
252106,air_eca5e0064dc9314a,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.419278,0.0,21,23.090765,0.0,19.785714,0.0,21.761905,0.0
252107,air_ef47430bcd6f6a89,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.421763,0.0,10,23.091059,0.0,19.000000,0.0,23.113821,0.0
252108,air_00a91d42b08b08d9,2017-04-23,Sunday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2017,4,...,0.0,16.960526,0.0,1,20.361010,0.0,54.000000,0.0,22.493506,0.0


In [28]:
data

,store_id,date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,...,is_nan_lag_holiday_flg_273,lag_area_genre_day_of_week_365,is_nan_lag_area_genre_day_of_week_365,visitors,lag_area_genre_holiday_flg_365,is_nan_lag_area_genre_holiday_flg_365,lag_day_of_week_365,is_nan_lag_day_of_week_365,lag_holiday_flg_365,is_nan_lag_holiday_flg_365
0,air_00a91d42b08b08d9,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,...,1.0,0.000000,1.0,24,0.000000,1.0,0.000000,1.0,0.000000,1.0
1,air_0241aa3964b7f861,2016-01-03,Sunday,1,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996,2016,1,...,1.0,0.000000,1.0,28,0.000000,1.0,0.000000,1.0,0.000000,1.0
2,air_034a3d5b40d5b1b1,2016-07-01,Friday,0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229,2016,7,...,1.0,0.000000,1.0,9,0.000000,1.0,0.000000,1.0,0.000000,1.0
3,air_036d4f1ee7285390,2016-07-01,Friday,0,Cafe/Sweets,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073,2016,7,...,1.0,0.000000,1.0,10,0.000000,1.0,0.000000,1.0,0.000000,1.0
4,air_04341b588bde96cd,2016-01-01,Friday,1,Izakaya,Tōkyō-to Nerima-ku Toyotamakita,35.735623,139.651658,2016,1,...,1.0,0.000000,1.0,12,0.000000,1.0,0.000000,1.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252104,air_e57dd6884595f60d,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.414295,0.0,47,23.090176,0.0,24.730769,0.0,23.020690,0.0
252105,air_eb120e6d384a17a8,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.416789,0.0,35,23.090470,0.0,23.285714,0.0,22.698052,0.0
252106,air_eca5e0064dc9314a,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.419278,0.0,21,23.090765,0.0,19.785714,0.0,21.761905,0.0
252107,air_ef47430bcd6f6a89,2017-04-22,Saturday,0,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,2017,4,...,0.0,22.421763,0.0,10,23.091059,0.0,19.000000,0.0,23.113821,0.0


In [52]:
%prun submission.progress_apply(lambda row: make_prediction(row), axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

KeyError: "['visit_date'] not in index"

In [24]:
%prun submission.progress_apply(lambda row: make_prediction(row), axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

         26963859 function calls (26316516 primitive calls) in 58.270 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10142    7.911    0.001   12.350    0.001 managers.py:2276(_merge_blocks)
     1800    4.280    0.002    5.654    0.003 concat.py:94(concatenate_managers)
     4114    4.272    0.001    4.299    0.001 shape_base.py:219(vstack)
    89342    3.655    0.000    3.655    0.000 {method 'copy' of 'numpy.ndarray' objects}
5544047/5476912    1.395    0.000    2.254    0.000 {built-in method builtins.isinstance}
     5700    1.329    0.000    1.364    0.000 array_ops.py:113(comp_method_OBJECT_ARRAY)
      100    0.889    0.009   57.732    0.577 2868219650.py:32(make_prediction)
   101437    0.757    0.000    0.757    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    50516    0.646    0.000    1.473    0.000 take.py:120(_take_nd_ndarray)
1616485/1098981    0.539    0.000    0.738    0.000 {built-in method builti

In [13]:
%prun

ID_SIZE = 20

submission = pd.read_csv("data/raw/sample_submission.csv")

store_info = pd.read_csv("data/raw/air_store_info.csv")
store_info = store_info.rename(
    columns={
        "air_store_id": "store_id",
        "air_genre_name": "genre_name",
        "air_area_name": "area_name",
    }   
)

date_info = pd.read_csv("data/raw/date_info.csv")
date_info = date_info.rename(columns={"calendar_date": "date"})
date_info["date"] = date_info["date"].astype("string")

data = pd.read_csv("data/processed/data.csv")
data.head()

data["date"] = pd.to_datetime(data["date"])
data = data.rename(columns={"date": "visit_date"})

X = data.drop(columns=["visitors"]).reset_index(drop=True)
num_cols = np.array(X.select_dtypes(include=['number']).columns)

scaler = StandardScaler()
scaler.fit(X[num_cols])

def make_prediction(row):
    global data
    new_row = pd.DataFrame(row).T
    new_row["store_id"] = row["id"][:ID_SIZE]
    new_row["date"] = row["id"][ID_SIZE + 1:]
    new_row = new_row.drop(columns="id")

    new_row = pd.merge(new_row, date_info, on="date")
    new_row = pd.merge(new_row, store_info, on="store_id", how="left")
    
    new_row["date"] = pd.to_datetime(new_row["date"])
    new_row["year"] = new_row["date"].dt.year
    new_row["month"] = new_row["date"].dt.month
    new_row["day"] = new_row["date"].dt.day

    data = pd.concat([data, new_row]).reset_index(drop=True)

    for lag in LAGS:
        data = get_area_genre_lag_last(data, lag, "day_of_week")
        data = get_area_genre_lag_last(data, lag, "holiday_flg")
        data = add_lag_last(data, lag, "day_of_week")
        data = add_lag_last(data, lag, "holiday_flg")


    new_row = data.tail(1)

    for c in new_row.columns:
        col_type = new_row[c].dtype
        if col_type == 'object' or col_type.name == 'category':
            new_row[c] = new_row[c].astype('category')

    new_row = new_row.drop(columns=["date", "visitors"])

    new_row[num_cols] = scaler.transform(new_row[num_cols])
    
    pred = 1
    # pred = model.predict(new_row)
    pred = max(0, pred)

    row["visitors"] = pred

    data.at[data.index[-1], "visitors"] = pred
   
    return row



tqdm.pandas()
submission = submission.head(100)
submission = submission.progress_apply(lambda row: make_prediction(row), axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:store_id: category, visit_date: datetime64[ns], day_of_week: category, genre_name: category, area_name: category

         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)

In [15]:
directory = "data/submissions"
os.makedirs(directory, exist_ok=True)

submission.to_csv(directory + "/xgboost.csv", index=False)